In [63]:
from bs4 import BeautifulSoup
import requests
import re

In [64]:
import pandas as pd
import numpy as np

In [65]:
def simple_parsing(url):
    r  = requests.get(url)
    data = r.text
    return BeautifulSoup(data, "html.parser")

In [66]:
prefix = 'https://he.wikisource.org'
OC_url = prefix + '/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%90%D7%95%D7%A8%D7%97_%D7%97%D7%99%D7%99%D7%9D'
YD_url = prefix + '/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%99%D7%95%D7%A8%D7%94_%D7%93%D7%A2%D7%94'
EH_url = prefix + '/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%90%D7%91%D7%9F_%D7%94%D7%A2%D7%96%D7%A8'
CM_url = prefix + '/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98'

## OC

In [86]:
soup = simple_parsing(OC_url)
simanim = soup.findAll('p')[14]
links = soup.findAll('a', text=simanim)

In [78]:
df = pd.DataFrame([i.text for i in soup.findAll('a', text=simanim)])
df = df[0].str.split('•', expand=True)
df['Section'] = 'OC'
df = df.rename(columns={0:'Number', 1:'Description', 2:'Length'})
df.head()

,Number,Description,Length,Section
0,,None,None,OC
1,סימן א',דין השכמת הבוקר,ובו ט' סעיפים,OC
2,סימן ב',דין לבישת בגדים,ובו ו' סעיפים,OC
3,סימן ג',דין הנהגת בית הכסא,"ובו י""ז סעיפים",OC
4,סימן ד',דיני נטילת ידים,"ובו כ""ג סעיפים",OC


In [79]:
df.tail()

,Number,Description,Length,Section
696,"סימן תרצ""ו",דיני הספד ותענית ועשיית מלאכה בפורים,ובו ח' סעיפים,OC
697,"סימן תרצ""ז",אין אומרים תחינה בארבעה עשר וחמישה עשר שבאדר ...,ובו סעיף אחד,OC
698,,None,None,OC
699,,None,None,OC
700,,None,None,OC


In [80]:
df = df.drop(index=[0, 698, 699, 700])

Strip leading/trailing whitespace

In [81]:
for i in list(df):
    df[i] = df[i].str.strip()

In [82]:
for i in df.Length.unique():
    print(i)

ובו ט' סעיפים
ובו ו' סעיפים
ובו י"ז סעיפים
ובו כ"ג סעיפים
ובו סעיף אחד
ובו ד' סעיפים
ובו י"ב סעיפים
ובו ט"ו סעיפים
ובו ג' סעיפים
ובו ה' סעיפים
ובו ב' סעיפים
ובו י"ג סעיפים
ובו י"א סעיפים
ובו נ"ב סעיפים
ובו י' סעיפים
ובו ח' סעיפים
ובו י"ד סעיפים
ובו כ"ו סעיפים
ובו כ"ב סעיפים
ובו ז' סעיפים
ובו כ"ז סעיפים
ובו מ"ה סעיפים
ובו כ' סעיפים
ובו י"ח סעיפים
ובו נ"א סעיפים
ובו י"ט סעיפים
ובו ט"ז סעיפים
ובו מ"ט סעיפים
ובו ה' הסעיפים
ובו ב' בסעיפים
ובו ל"ו סעיפים
ובו כ"ד סעיפים
ובו כ"ה סעיפים


In [83]:
df['Seifim'] = df.Length  # backup
swap=dict(zip(df.Length.unique(),[9,6,17,23,1,4,12,15,3,5,2,13,11,52,10,8,14,26,22,7,27,45,20,18,51,19,16,49,5,2,36,24,25]))
df.Length = df.Length.map(swap)

In [84]:
df.head()

,Number,Description,Length,Section,Seifim
1,סימן א',דין השכמת הבוקר,9,OC,ובו ט' סעיפים
2,סימן ב',דין לבישת בגדים,6,OC,ובו ו' סעיפים
3,סימן ג',דין הנהגת בית הכסא,17,OC,"ובו י""ז סעיפים"
4,סימן ד',דיני נטילת ידים,23,OC,"ובו כ""ג סעיפים"
5,סימן ה',דין כוונת הברכות,1,OC,ובו סעיף אחד


In [88]:
siman_number = 1
siman_to_siman_lengths = {}
for link in links[1:-3]:
    siman_url = prefix + link.attrs['href']
    print('Retrieving {}'.format(siman_number))
    soup = simple_parsing(siman_url)
    siman_to_siman_lengths[siman_number] = len(soup.findAll("span", id=re.compile('סעיף.*')))
    siman_number += 1

Retrieving 1
Retrieving 2
Retrieving 3
Retrieving 4
Retrieving 5
Retrieving 6
Retrieving 7
Retrieving 8
Retrieving 9
Retrieving 10
Retrieving 11
Retrieving 12
Retrieving 13
Retrieving 14
Retrieving 15
Retrieving 16
Retrieving 17
Retrieving 18
Retrieving 19
Retrieving 20
Retrieving 21
Retrieving 22
Retrieving 23
Retrieving 24
Retrieving 25
Retrieving 26
Retrieving 27
Retrieving 28
Retrieving 29
Retrieving 30
Retrieving 31
Retrieving 32
Retrieving 33
Retrieving 34
Retrieving 35
Retrieving 36
Retrieving 37
Retrieving 38
Retrieving 39
Retrieving 40
Retrieving 41
Retrieving 42
Retrieving 43
Retrieving 44
Retrieving 45
Retrieving 46
Retrieving 47
Retrieving 48
Retrieving 49
Retrieving 50
Retrieving 51
Retrieving 52
Retrieving 53
Retrieving 54
Retrieving 55
Retrieving 56
Retrieving 57
Retrieving 58
Retrieving 59
Retrieving 60
Retrieving 61
Retrieving 62
Retrieving 63
Retrieving 64
Retrieving 65
Retrieving 66
Retrieving 67
Retrieving 68
Retrieving 69
Retrieving 70
Retrieving 71
Retrieving 72
R

Retrieving 555
Retrieving 556
Retrieving 557
Retrieving 558
Retrieving 559
Retrieving 560
Retrieving 561
Retrieving 562
Retrieving 563
Retrieving 564
Retrieving 565
Retrieving 566
Retrieving 567
Retrieving 568
Retrieving 569
Retrieving 570
Retrieving 571
Retrieving 572
Retrieving 573
Retrieving 574
Retrieving 575
Retrieving 576
Retrieving 577
Retrieving 578
Retrieving 579
Retrieving 580
Retrieving 581
Retrieving 582
Retrieving 583
Retrieving 584
Retrieving 585
Retrieving 586
Retrieving 587
Retrieving 588
Retrieving 589
Retrieving 590
Retrieving 591
Retrieving 592
Retrieving 593
Retrieving 594
Retrieving 595
Retrieving 596
Retrieving 597
Retrieving 598
Retrieving 599
Retrieving 600
Retrieving 601
Retrieving 602
Retrieving 603
Retrieving 604
Retrieving 605
Retrieving 606
Retrieving 607
Retrieving 608
Retrieving 609
Retrieving 610
Retrieving 611
Retrieving 612
Retrieving 613
Retrieving 614
Retrieving 615
Retrieving 616
Retrieving 617
Retrieving 618
Retrieving 619
Retrieving 620
Retrieving

In [89]:
df['Scraped_num'] = siman_to_siman_lengths.values()

In [90]:
check_df = df[df['Length'] != 0]
check_df[check_df.Length != check_df.Scraped_num]

,Number,Description,Length,Section,Seifim,Scraped_num
23,"סימן כ""ג",דיני ציצית בבית הקברות,3,OC,ובו ג' סעיפים,4
106,"סימן ק""ו",מי הם הפטורים מתפילה,3,OC,ובו ג' סעיפים,2
117,"סימן קי""ז",דיני ברכת השנים,8,OC,ובו ח' סעיפים,5
253,"סימן רנ""ג","דין כירה ותנור, ליתן עליה הקדירות בערב שבת",8,OC,ובו ח' סעיפים,5


In [91]:
df = df.rename(columns={'Length':'Old_length', 'Scraped_num': 'Length'})

In [92]:
df.Length.sum()

4169

## YD

In [104]:
soup = simple_parsing(YD_url)
simanim = soup.findAll('p')[13]
links = soup.findAll('a', text=simanim)

In [105]:
df = pd.DataFrame([i.text for i in soup.findAll('a', text=simanim)])
df = df[0].str.split('•', expand=True)
df['Section'] = 'YD'
df = df.rename(columns={0:'Number', 1:'Description', 2:'Length'})
df.head()

,Number,Description,Length,Section
0,,None,None,YD
1,סימן א',מי הם הכשרים לשחוט,"ובו י""ד סעיפים",YD
2,סימן ב',אם שחיטת עובד כוכבים ומומר כשרה,"ובו י""א סעיפים",YD
3,סימן ג',שחיטה אינה צריכה כוונה,ובו סעיף אחד,YD
4,סימן ד',השוחט לשם עבודת כוכבים או לשם דבר אחר מה דינו,ובו ז' סעיפים,YD


In [106]:
df.tail()

,Number,Description,Length,Section
402,"סימן ת""ב",דין שמועה קרובה ורחוקה,ובו ? סעיפים,YD
403,"סימן ת""ג",דיני המלקט עצמות,ובו ? סעיפים,YD
404,,None,None,YD
405,,None,None,YD
406,,None,None,YD


In [107]:
df = df.drop(index=[0, 404, 405, 406])

In [108]:
for i in list(df):
    df[i] = df[i].str.strip()

In [109]:
for i in df.Length.unique():
    print(i)

ובו י"ד סעיפים
ובו י"א סעיפים
ובו סעיף אחד
ובו ז' סעיפים
ובו ג' סעיפים
ובו ד' סעיפים
ובו ב' סעיפים
ובו ו' סעיפים
ובו י"ב סעיפים
ובו כ' סעיפים
ובו ח' סעיפים
ובו ה' סעיפים
ובו כ"ד סעיפים
ובו י' סעיפים
ובו י"ז סעיפים
ובו כ"ה סעיפים
ובו ט' סעיפים
ובו י"ג סעיפים
ובו כ"א סעיפים
ובו ל"ג סעיפים
ובו ט"ו סעיפים
ובו ט"ז סעיפים
ובו כ"ו סעיפים
ובו כ"ג סעיפים
ובו כ"ז סעיפים
ובו י"ט סעיפים
ובו מ' סעיפים
ובו י"טסעיפים
ובו ל"ד סעיפים
ובו נ"ד סעיפים
ובו מ"ח סעיפים
ובו ע"ה סעיפים
ובו נ"א סעיפים
ובו ע"ב סעיפים
ובו ? סעיפים
ובו יג סעיפים
ובו ג סעיפים


In [110]:
df['Seifim'] = df.Length  # backup
swap=dict(zip(df.Length.unique(),[14,11,1,7,3,4,2,6,12,20,8,5,24,10,17,25,9,13,21,33,15,16,26,23,27,19,40,19,34,54,48,75,51,72,0,13,3]))
df.Length = df.Length.map(swap)

In [111]:
df.head()

,Number,Description,Length,Section,Seifim
1,סימן א',מי הם הכשרים לשחוט,14,YD,"ובו י""ד סעיפים"
2,סימן ב',אם שחיטת עובד כוכבים ומומר כשרה,11,YD,"ובו י""א סעיפים"
3,סימן ג',שחיטה אינה צריכה כוונה,1,YD,ובו סעיף אחד
4,סימן ד',השוחט לשם עבודת כוכבים או לשם דבר אחר מה דינו,7,YD,ובו ז' סעיפים
5,סימן ה',השוחט לשם קדשים מה דינו,3,YD,ובו ג' סעיפים


We have mapped ? to 0, so we can get an interim total, but it is missing a lot:

In [112]:
df.Length.sum()

2234

Relevant links are from index 1 to -4. This cell scrapes over 400 pages, so it takes a couple of minutes - **try not to run it too much**!

In [113]:
siman_number = 1
siman_to_siman_lengths = {}
for link in links[1:-3]:
    siman_url = prefix + link.attrs['href']
    print('Retrieving {}'.format(siman_number))
    soup = simple_parsing(siman_url)
    siman_to_siman_lengths[siman_number] = len(soup.findAll("span", id=re.compile('סעיף.*')))
    siman_number += 1

Retrieving 1
Retrieving 2
Retrieving 3
Retrieving 4
Retrieving 5
Retrieving 6
Retrieving 7
Retrieving 8
Retrieving 9
Retrieving 10
Retrieving 11
Retrieving 12
Retrieving 13
Retrieving 14
Retrieving 15
Retrieving 16
Retrieving 17
Retrieving 18
Retrieving 19
Retrieving 20
Retrieving 21
Retrieving 22
Retrieving 23
Retrieving 24
Retrieving 25
Retrieving 26
Retrieving 27
Retrieving 28
Retrieving 29
Retrieving 30
Retrieving 31
Retrieving 32
Retrieving 33
Retrieving 34
Retrieving 35
Retrieving 36
Retrieving 37
Retrieving 38
Retrieving 39
Retrieving 40
Retrieving 41
Retrieving 42
Retrieving 43
Retrieving 44
Retrieving 45
Retrieving 46
Retrieving 47
Retrieving 48
Retrieving 49
Retrieving 50
Retrieving 51
Retrieving 52
Retrieving 53
Retrieving 54
Retrieving 55
Retrieving 56
Retrieving 57
Retrieving 58
Retrieving 59
Retrieving 60
Retrieving 61
Retrieving 62
Retrieving 63
Retrieving 64
Retrieving 65
Retrieving 66
Retrieving 67
Retrieving 68
Retrieving 69
Retrieving 70
Retrieving 71
Retrieving 72
R

In [114]:
df['Scraped_num'] = siman_to_siman_lengths.values()

Find errors

In [115]:
check_df = df[df['Length'] != 0]
check_df[check_df.Length != check_df.Scraped_num]

,Number,Description,Length,Section,Seifim,Scraped_num
124,"סימן קכ""ד","מי הוא העושה יין נסך, ודיני נגיעת הגוי בחבית",26,YD,"ובו כ""ו סעיפים",27
233,"סימן רל""ד",מתי האב והבעל מפירין ומתי מפירין שניהם יחד,72,YD,"ובו ע""ב סעיפים",74


There are two mistakes, in both the scraper is right!<br>(Numbered index is one off of most of them because of siman 168-169 being combined...)

In [116]:
df.iloc[166:169,:]

,Number,Description,Length,Section,Seifim,Scraped_num
167,"סימן קס""ז",באיזה אופן מותר להלוות מעות בתנאי שיתעסק בו לר...,1,YD,ובו סעיף אחד,1
168,"סימן קס""ח-קס""ט",כמה דיני ריבית הנעשות באמצעות הגוי,27,YD,"ובו כ""ז סעיפים",27
169,"סימן ק""ע",שלא יוכל ישראל להיות ערב לישראל שלווה מן הגוי ...,2,YD,ובו ב' סעיפים,2


In [117]:
df = df.rename(columns={'Length':'Old_length', 'Scraped_num': 'Length'})

In [118]:
df.Length.sum()

3701

## EH

In [143]:
soup = simple_parsing(EH_url)
simanim = soup.findAll('p')[9]
links = soup.findAll('a', text=simanim)

In [144]:
df = pd.DataFrame([i.text for i in soup.findAll('a', text=simanim)])
df = df[0].str.split('•', expand=True)
df['Section'] = 'EH'
df = df.rename(columns={0:'Number', 1:'Description', 2:'Length'})
df.head()

,Number,Description,Length,Section
0,,None,None,EH
1,סימן א,דיני פריה ורביה ושלא לעמוד בלא אשה,"ובו י""ד סעיפים",EH
2,סימן ב,שישתדל כל אדם ליקח לו אשה הגונה,"ובו י""א סעיפים",EH
3,סימן ג,דין בן שהוא ספק לכהונה,ובו ט' סעיפים,EH
4,סימן ד,מי הם אסורים לבא בקהל ואיזה הנקרא ממזר,"ובו ל""ז סעיפים",EH


In [145]:
df = df.drop(index=[0, 181, 182, 183])

Two extra simanim - סדר הגט, and סדר חליצה 

In [146]:
df.iloc[153:156,:]

,Number,Description,Length,Section
154,סימן קנד,"למי שכופין להוציא בגט, בין איש בין אשה, וסדר ...",ובו ? סעיפים,EH
155,סדר הגט,"ובו דיני העדים והבעל והסופר והאשה, וכל פרטיו",ובו ? סעיפים,EH
156,סימן קנה,"עד איזה זמן הקטנה ממאנת, ויתר דיני מיאון",ובו ? סעיפים,EH


In [147]:
df.iloc[169:172,:]

,Number,Description,Length,Section
170,סימן קסט,כל דיני חליצה בפרטות,ובו ? סעיפים,EH
171,סדר חליצה,ובו ? סעיפים,None,EH
172,סימן קע,דין גט אחר גט ומאמר אחר מאמר,ובו ? סעיפים,EH


In [148]:
for i in list(df):
    df[i] = df[i].str.strip()

In [149]:
for i in df.Length.unique():
    print(i)

ובו י"ד סעיפים
ובו י"א סעיפים
ובו ט' סעיפים
ובו ל"ז סעיפים
ובו ט"ו סעיפים
ובו כ"ג סעיפים
ובו ה' סעיפים
ובו ב' סעיפים
ובו ז' סעיפים
ובו ח' סעיפים
ובו ד' סעיפים
ובו ? סעיפים
None


In [150]:
df['Seifim'] = df.Length  # backup
swap=dict(zip(df.Length.unique(),[14,11,9,37,15,23,5,2,7,8,4,0,0]))
df.Length = df.Length.map(swap)

In [151]:
siman_number = 1
siman_to_siman_lengths = {}
for link in links[1:-3]:
    siman_url = prefix + link.attrs['href']
    print('Retrieving {}'.format(siman_number))
    soup = simple_parsing(siman_url)
    siman_to_siman_lengths[siman_number] = len(soup.findAll("span", id=re.compile('סעיף.*')))
    siman_number += 1

Retrieving 1
Retrieving 2
Retrieving 3
Retrieving 4
Retrieving 5
Retrieving 6
Retrieving 7
Retrieving 8
Retrieving 9
Retrieving 10
Retrieving 11
Retrieving 12
Retrieving 13
Retrieving 14
Retrieving 15
Retrieving 16
Retrieving 17
Retrieving 18
Retrieving 19
Retrieving 20
Retrieving 21
Retrieving 22
Retrieving 23
Retrieving 24
Retrieving 25
Retrieving 26
Retrieving 27
Retrieving 28
Retrieving 29
Retrieving 30
Retrieving 31
Retrieving 32
Retrieving 33
Retrieving 34
Retrieving 35
Retrieving 36
Retrieving 37
Retrieving 38
Retrieving 39
Retrieving 40
Retrieving 41
Retrieving 42
Retrieving 43
Retrieving 44
Retrieving 45
Retrieving 46
Retrieving 47
Retrieving 48
Retrieving 49
Retrieving 50
Retrieving 51
Retrieving 52
Retrieving 53
Retrieving 54
Retrieving 55
Retrieving 56
Retrieving 57
Retrieving 58
Retrieving 59
Retrieving 60
Retrieving 61
Retrieving 62
Retrieving 63
Retrieving 64
Retrieving 65
Retrieving 66
Retrieving 67
Retrieving 68
Retrieving 69
Retrieving 70
Retrieving 71
Retrieving 72
R

In [152]:
df['Scraped_num'] = siman_to_siman_lengths.values()

In [153]:
check_df = df[df['Length'] != 0]
check_df[check_df.Length != check_df.Scraped_num]

,Number,Description,Length,Section,Seifim,Scraped_num
6,סימן ו,נשים האסורות לכהן ודין אשת כהן,15,EH,"ובו ט""ו סעיפים",18


Scraper is correct!

In [154]:
df = df.rename(columns={'Length':'Old_length', 'Scraped_num': 'Length'})

In [155]:
df.Length.sum()

1988

## CM

In [181]:
soup = simple_parsing(CM_url)
simanim = soup.findAll('a')
simanim = simanim[80:507]

In [182]:
df = pd.DataFrame([i.attrs['title'] for i in simanim])  # page is formatted slightly differently
df['Section'] = 'CM'
df = df.rename(columns={0:'Number'})

In [183]:
for i in list(df):
    df[i] = df[i].str.strip()

In [184]:
siman_number = 1
siman_to_siman_lengths = {}
for siman in simanim:
    siman_url = prefix + siman.attrs['href']
    print('Retrieving {}'.format(siman_number))
    soup = simple_parsing(siman_url)
    siman_to_siman_lengths[siman_number] = len(soup.findAll("span", id=re.compile('סעיף.*')))
    siman_number += 1

Retrieving 1
Retrieving 2
Retrieving 3
Retrieving 4
Retrieving 5
Retrieving 6
Retrieving 7
Retrieving 8
Retrieving 9
Retrieving 10
Retrieving 11
Retrieving 12
Retrieving 13
Retrieving 14
Retrieving 15
Retrieving 16
Retrieving 17
Retrieving 18
Retrieving 19
Retrieving 20
Retrieving 21
Retrieving 22
Retrieving 23
Retrieving 24
Retrieving 25
Retrieving 26
Retrieving 27
Retrieving 28
Retrieving 29
Retrieving 30
Retrieving 31
Retrieving 32
Retrieving 33
Retrieving 34
Retrieving 35
Retrieving 36
Retrieving 37
Retrieving 38
Retrieving 39
Retrieving 40
Retrieving 41
Retrieving 42
Retrieving 43
Retrieving 44
Retrieving 45
Retrieving 46
Retrieving 47
Retrieving 48
Retrieving 49
Retrieving 50
Retrieving 51
Retrieving 52
Retrieving 53
Retrieving 54
Retrieving 55
Retrieving 56
Retrieving 57
Retrieving 58
Retrieving 59
Retrieving 60
Retrieving 61
Retrieving 62
Retrieving 63
Retrieving 64
Retrieving 65
Retrieving 66
Retrieving 67
Retrieving 68
Retrieving 69
Retrieving 70
Retrieving 71
Retrieving 72
R

In [185]:
df['Scraped_num'] = siman_to_siman_lengths.values()
df = df.rename(columns={'Scraped_num':'Length'})
df.Length.sum()

3692

In [186]:
volumes = ['OC.txt', 'YD.txt', 'EH.txt', 'CM.txt']
lengths = {}

In [187]:
for volume in volumes:
    with open(volume, "r", encoding='utf-8') as f:
        merged = f.read()
        lengths[volume] = [__ for __ in [_ for _ in merged.split('\n')[merged.split('\n').index('Siman 1'):] if _]]

In [189]:
simanim = {}
count = 0
for item in lengths['CM.txt'][::-1]:
    if 'Siman ' in item:
        simanim[item] = count
        count = 0
    else:
        count += 1

In [190]:
temp = {k:simanim[k] for k in list(simanim.keys())[::-1]}

For YD: Siman 297 was doubled

In [ ]:
df.loc[296,'Length'] = 56
df = df.drop(297)

In [132]:
temp.pop('Siman 169')

27

End YD

In [191]:
len(temp.values())

427

In [166]:
df.iloc[153:156,:]

,Number,Description,Old_length,Section,Seifim,Length
154,סימן קנד,"למי שכופין להוציא בגט, בין איש בין אשה, וסדר כ...",0,EH,ובו ? סעיפים,24
155,סדר הגט,"ובו דיני העדים והבעל והסופר והאשה, וכל פרטיו",0,EH,ובו ? סעיפים,101
156,סימן קנה,"עד איזה זמן הקטנה ממאנת, ויתר דיני מיאון",0,EH,ובו ? סעיפים,22


In [167]:
df.iloc[169:172,:]

,Number,Description,Old_length,Section,Seifim,Length
170,סימן קסט,כל דיני חליצה בפרטות,0,EH,ובו ? סעיפים,56
171,סדר חליצה,ובו ? סעיפים,0,EH,None,57
172,סימן קע,דין גט אחר גט ומאמר אחר מאמר,0,EH,ובו ? סעיפים,20


In [165]:
temp.values()

dict_values([14, 11, 9, 37, 14, 18, 23, 5, 2, 7, 8, 4, 14, 1, 31, 6, 58, 1, 2, 2, 7, 20, 7, 1, 10, 4, 10, 23, 10, 11, 9, 4, 2, 4, 15, 12, 27, 39, 7, 8, 4, 5, 2, 12, 3, 8, 4, 7, 3, 7, 1, 1, 3, 1, 7, 4, 1, 2, 4, 1, 2, 13, 2, 6, 4, 13, 11, 10, 7, 12, 4, 1, 9, 12, 5, 13, 5, 8, 3, 18, 2, 8, 2, 1, 19, 2, 2, 12, 4, 20, 5, 8, 32, 7, 7, 21, 2, 7, 2, 16, 4, 9, 8, 6, 7, 2, 10, 3, 4, 2, 17, 18, 10, 12, 10, 6, 11, 19, 11, 11, 8, 2, 5, 9, 23, 50, 13, 7, 34, 22, 9, 4, 3, 10, 5, 7, 5, 4, 18, 11, 69, 19, 23, 7, 10, 5, 3, 2, 7, 4, 1, 12, 1, 24, 22, 13, 10, 6, 7, 8, 9, 5, 8, 8, 5, 9, 6, 9, 56, 20, 10, 16, 17, 7, 14, 8, 5, 183])

In [175]:
eh_simanim = (list(temp.values())[:-24] + [101] + list(temp.values())[-24:-9] + [57] + list(temp.values())[-9:])

In [193]:
df['Sefaria'] = list(temp.values())

In [194]:
df[df['Length'] != df['Sefaria']]

,Number,Section,Length,Sefaria
40,שולחן ערוך חושן משפט מא,CM,4,5
73,שולחן ערוך חושן משפט עד,CM,7,8
75,שולחן ערוך חושן משפט עו,CM,3,4


In [195]:
df_CM = df
df_CM.to_csv('CM.csv')